# Sample GeoEDF Hydrology Workflow

This notebook demonstrates a GeoEDF hydrologic workflow that acquires data (in HDF format) from a NASA Distributed Active Archive Center (DAAC) and aggregates the data across a provided watershed region. This is often the first step before running a hydrologic model. 

![Workflow](img/research.png)

In GeoEDF, this workflow combines a data connector (**NASAInput**) and processor (**HDFEOSShapefileMask**).

## Step 1: Run a workflow consisting of pre-existing GeoEDF plugins

Take a look at the file ``/home/scitech/shared-data/sample-geoedf-wf/workflow/mcd15-simple.yml``. This workflow will be processed and executed using GeoEDF.

![Workflow](img/mcd.png)

In [ ]:
from geoedfengine.GeoEDFWorkflow import GeoEDFWorkflow

In [ ]:
workflow = GeoEDFWorkflow('/home/scitech/shared-data/sample-geoedf-wf/workflow/mcd15-simple.yml')

In [ ]:
workflow.execute()